In [4]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Đọc dữ liệu từ tệp CSV
data = pd.read_csv('real_estate_processed.csv')

# Lựa chọn các đặc trưng và biến mục tiêu
X = data[['size', 'rooms', 'category_name', 'area_name', 'street_name']]
y = data['price']

# Phân chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chọn các đặc trưng cho việc phân cụm
X_cluster = X_train[['size', 'rooms']]

# Huấn luyện mô hình KMeans
kmeans = KMeans(n_clusters=2)
kmeans.fit(X_cluster)

# Lấy nhãn của các cụm cho tập huấn luyện
cluster_labels_train = kmeans.predict(X_cluster)

# Tính giá trị trung bình của các mẫu trong mỗi cụm
cluster_centers = kmeans.cluster_centers_

# Dự đoán giá trung bình cho mỗi cụm
cluster_prices = []
for label in np.unique(cluster_labels_train):
    cluster_prices.append(y_train[cluster_labels_train == label].mean())

# Tạo một bảng dữ liệu mới chứa thông tin về cụm và giá trị dự đoán tương ứng
cluster_data_train = pd.DataFrame({'cluster_label': np.unique(cluster_labels_train), 'cluster_price': cluster_prices})

# Thêm cột cluster_label vào tập kiểm tra
X_test['cluster_label'] = kmeans.predict(X_test[['size', 'rooms']])

# Merge với bảng cluster_data_train để lấy giá trị dự đoán cho mỗi cụm
X_test_merged = pd.merge(X_test, cluster_data_train, on='cluster_label', how='left')

# Sử dụng Linear Regression để dự đoán giá dựa trên đặc trưng và giá trị dự đoán từ KMeans
model = LinearRegression()
model.fit(X_test_merged.drop(columns=['cluster_label']), y_test)

# Đánh giá mô hình trên tập kiểm tra
test_score = model.score(X_test_merged.drop(columns=['cluster_label']), y_test)

print("R^2 Score on Test Data:", test_score)


C:\Users\Nhan\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


R^2 Score on Test Data: 0.2885983995212549
